In [ ]:
def load_input_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    initial_capital = float(lines[0].split(":")[-1].strip().replace("$", ""))
    generations = int(lines[1].split(":")[-1].strip())
    historical_prices = list(map(float, lines[2].split(":")[-1].strip().strip("[]").split(",")))
    population = []
    for line in lines[3:]:
        if "{" in line:
            values = list(map(float, line.replace("{" , "").replace("}", "").replace("\"", "").split(",")))
            population.append({'stop_loss': values[0], 'take_profit': values[1], 'trade_size': values[2]})

    return initial_capital, generations, historical_prices, population

def evaluate_fitness(chromosome, historical_prices, initial_capital):
    capital = initial_capital
    for change in historical_prices:
        trade_size = (chromosome['trade_size'] / 100) * capital
        if change <= -chromosome['stop_loss']:
            capital -= trade_size * (chromosome['stop_loss'] / 100)
        elif change >= chromosome['take_profit']:
            capital += trade_size * (chromosome['take_profit'] / 100)
        else:
            capital += trade_size * (change / 100)
    return capital - initial_capital

def select_parents(population, fitness_scores):
    return random.choices(population, weights=fitness_scores, k=2)

def crossover(parent1, parent2):
    crossover_point = random.randint(1, 2)
    child1 = {**parent1}
    child2 = {**parent2}
    keys = list(parent1.keys())
    for i in range(crossover_point, len(keys)):
        child1[keys[i]], child2[keys[i]] = child2[keys[i]], child1[keys[i]]
    return child1, child2

def mutate(chromosome, mutation_rate=0.05):
    if random.random() < mutation_rate:
        key = random.choice(list(chromosome.keys()))
        chromosome[key] = max(1, min(99, chromosome[key] + random.randint(-2, 2)))

def genetic_algorithm(input_file):
    initial_capital, generations, historical_prices, population = load_input_file(input_file)

    for _ in range(generations):
        fitness_scores = [evaluate_fitness(chromo, historical_prices, initial_capital) for chromo in population]
        new_population = []
        for _ in range(len(population) // 2):
            parent1, parent2 = select_parents(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1)
            mutate(child2)
            new_population.extend([child1, child2])
        population = sorted(new_population, key=lambda x: evaluate_fitness(x, historical_prices, initial_capital), reverse=True)[:len(population)]

    best_strategy = max(population, key=lambda x: evaluate_fitness(x, historical_prices, initial_capital))
    final_profit = evaluate_fitness(best_strategy, historical_prices, initial_capital)

    return {'best_strategy': best_strategy, 'Final_profit': final_profit}